## req 

In [11]:
pip install -U chromadb langchain llama-index langchain_experimental langchain_openai


Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
print(os.getcwd())  # This shows your current working directory


C:\Users\vreddy_quantum-i\Desktop\Symptoms_checker


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install pymupdf langchain


In [14]:
!pip install langchain openai tiktoken
!pip install faiss-cpu  # or faiss-gpu if needed
!pip install sentence-transformers
!pip install chromadb  # optional if using Chroma instead of FAISS
!pip install pymupdf  # for PDF reading
!pip install pypdf    # optional PDF reading alternative
!pip install python-dotenv  # to manage API keys

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [20]:
!pip install PyPDF2


## code

In [15]:
!pip install faiss-cpu

In [16]:
import fitz  # PyMuPDF

def extract_text(path):
    with fitz.open(path) as doc:  # ensures proper file closing
        return "\n".join([page.get_text() for page in doc])

# ✅ Make sure the filename has the correct extension
text = extract_text(r"C:\Users\vreddy_quantum-i\Downloads\DSM-5_filtered.pdf")


In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# Convert to a Langchain Document
doc = Document(page_content=text, metadata={"source": "1100_pages.pdf"})

# Initialize splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,         # number of characters per chunk
    chunk_overlap=200        # slight overlap between chunks
)

chunks = splitter.split_documents([doc])
print(f"Total Chunks: {len(chunks)}")
print(chunks[0].page_content[:300])  # Preview first chunk
print

Total Chunks: 4175
SECTION II
Diagnostic Criteria and Codes
Neurodevelopmental Disorders
Schizophrenia Spectrum and Other Psychotic Disorders
Bipolar and Related Disorders
Depressive Disorders
Anxiety Disorders
Obsessive-Compulsive and Related Disorders
Trauma- and Stressor-Related Disorders
Dissociative Disorders
Som


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [18]:
from sentence_transformers import SentenceTransformer

import faiss

import pickle
 
# Load model

model = SentenceTransformer("all-MiniLM-L6-v2")
 
# Assume your chunks are in a list called `chunks`

# Example: chunks = ["Depression is defined as...", "Schizophrenia criteria include..."]
 
# Embed chunks

embeddings = model.encode(chunks, show_progress_bar=True)
 
# Create FAISS index

dimension = embeddings.shape[1]

index = faiss.IndexFlatL2(dimension)

index.add(embeddings)
 
# Save index and chunks

faiss.write_index(index, "dsm5_index.faiss")

with open("dsm5_chunks.pkl", "wb") as f:

    pickle.dump(chunks, f) 

Batches:   0%|          | 0/131 [00:00<?, ?it/s]

TypeError: 'Document' object is not subscriptable

In [ ]:
import numpy as np
 
# Load index and chunks

index = faiss.read_index("dsm5_index.faiss")

with open("dsm5_chunks.pkl", "rb") as f:

    chunks = pickle.load(f)
 
# User query

query = "Patient reports low mood, fatigue, and feelings of worthlessness."
 
# Embed the query

query_embedding = model.encode([query])
 
# Search top 5 similar

D, I = index.search(np.array(query_embedding), k=5)
 
# Retrieve matched chunks

retrieved_chunks = [chunks[i] for i in I[0]]

In [ ]:
import fitz  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import numpy as np
from transformers import pipeline
import torch
import os

# Step 1: Extract text from PDF
def extract_text(path):
    try:
        with fitz.open(path) as doc:
            return "\n".join([page.get_text() for page in doc])
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return None

# Step 2: Prepare documents and chunk them
def prepare_documents(pdf_path):
    text = extract_text(pdf_path)
    if not text:
        raise ValueError("Failed to extract text from PDF")
    
    doc = Document(page_content=text, metadata={"source": pdf_path})
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_documents([doc])
    print(f"Total Chunks: {len(chunks)}")
    print(f"First chunk preview: {chunks[0].page_content[:300]}")
    return chunks

# Step 3: Create and save embeddings
def create_embeddings(chunks, model_name="all-MiniLM-L6-v2", index_file="dsm5_index.faiss", chunks_file="dsm5_chunks.pkl"):
    try:
        model = SentenceTransformer(model_name)
        chunk_texts = [chunk.page_content for chunk in chunks]
        embeddings = model.encode(chunk_texts, show_progress_bar=True)
        
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)
        index.add(embeddings)
        
        faiss.write_index(index, index_file)
        with open(chunks_file, "wb") as f:
            pickle.dump(chunks, f)
        print(f"Saved index to {index_file} and chunks to {chunks_file}")
        return index, chunks
    except Exception as e:
        print(f"Error creating embeddings: {e}")
        return None, None

# Step 4: Load index and chunks
def load_index_and_chunks(index_file="dsm5_index.faiss", chunks_file="dsm5_chunks.pkl"):
    if os.path.exists(index_file) and os.path.exists(chunks_file):
        try:
            index = faiss.read_index(index_file)
            with open(chunks_file, "rb") as f:
                chunks = pickle.load(f)
            print(f"Loaded index from {index_file} and chunks from {chunks_file}")
            return index, chunks
        except Exception as e:
            print(f"Error loading index or chunks: {e}")
            return None, None
    return None, None

# Step 5: Retrieve relevant chunks
def retrieve_chunks(query, index, chunks, model, top_k=3):
    try:
        query_embedding = model.encode([query])[0]
        distances, indices = index.search(np.array([query_embedding]), top_k)
        return [chunks[i].page_content for i in indices[0]]
    except Exception as e:
        print(f"Error retrieving chunks: {e}")
        return []

# Step 6: Generate response
def generate_response(query, relevant_chunks, model_name="distilgpt2"):
    try:
        generator = pipeline("text-generation", model=model_name, device=0 if torch.cuda.is_available() else -1)
        context = "\n".join(relevant_chunks)
        prompt = f"Question: {query}\nContext: {context}\nAnswer in a concise and accurate manner:"
        response = generator(prompt, max_length=200, num_return_sequences=1, truncation=True)[0]["generated_text"]
        # Extract only the answer part
        answer = response.split("Answer:")[-1].strip() if "Answer:" in response else response.strip()
        return answer
    except Exception as e:
        print(f"Error generating response: {e}")
        return "Sorry, I couldn't generate a response."

# Main function
def main(pdf_path, query):
    # Prepare documents
    chunks = prepare_documents(pdf_path)
    if not chunks:
        return "Failed to prepare documents."
    
    # Load or create embeddings
    index_file, chunks_file = "dsm5_index.faiss", "dsm5_chunks.pkl"
    index, chunks = load_index_and_chunks(index_file, chunks_file)
    model = SentenceTransformer("all-MiniLM-L6-v2")
    
    if index is None or chunks is None:
        print("Creating new embeddings...")
        index, chunks = create_embeddings(chunks, model_name="all-MiniLM-L6-v2", index_file=index_file, chunks_file=chunks_file)
        if index is None or chunks is None:
            return "Failed to create embeddings."
    
    # Retrieve and generate
    relevant_chunks = retrieve_chunks(query, index, chunks, model, top_k=3)
    if not relevant_chunks:
        return "No relevant information found."
    
    response = generate_response(query, relevant_chunks)
    return response

# Command-line interface for testing
if __name__ == "__main__":
    pdf_path = r"C:\Users\vreddy_quantum-i\Downloads\DSM-5_filtered.pdf"
    
    # Verify PDF exists
    if not os.path.exists(pdf_path):
        print(f"PDF file not found at {pdf_path}")
    else:
        while True:
            query = input("Enter your question (or 'quit' to exit): ")
            if query.lower() == "quit":
                break
            response = main(pdf_path, query)
            print(f"\nQuery: {query}")
            print(f"Response: {response}\n")

### test code


In [24]:
# COMPLETE JUPYTER NOTEBOOK SOLUTION
# Copy each cell block into separate Jupyter notebook cells

# ===== CELL 1: Install Required Packages =====
# Run this first if packages are not installed
"""
!pip install pandas numpy scikit-learn sentence-transformers torch
!pip install PyPDF2 PyMuPDF langchain nltk
"""

# ===== CELL 2: Imports and Setup =====
import pandas as pd
import numpy as np
import pickle
import os
import re
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# PDF processing
import PyPDF2
import fitz  # PyMuPDF

# ML and embeddings
from sentence_transformers import SentenceTransformer
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Text processing
from langchain.text_splitter import RecursiveCharacterTextSplitter
import nltk

# Download required NLTK data
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
except:
    pass

print("✅ All imports successful!")

# ===== CELL 3: Define PDFSymptomCheckerRAG Class =====
class PDFSymptomCheckerRAG:
    def __init__(self, embedding_model='all-MiniLM-L6-v2', use_gpu=False):
        """
        Initialize the RAG system
        
        Args:
            embedding_model: Sentence transformer model to use
            use_gpu: Whether to use GPU for embeddings (if available)
        """
        self.embedding_model_name = embedding_model
        self.use_gpu = use_gpu and torch.cuda.is_available()
        
        # Initialize sentence transformer
        device = 'cuda' if self.use_gpu else 'cpu'
        print(f"Loading embedding model: {embedding_model} on {device}")
        self.embedder = SentenceTransformer(embedding_model, device=device)
        
        # Initialize text splitter for chunking
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
            separators=["\n\n", "\n", ". ", " ", ""]
        )
        
        # Storage
        self.chunks = []
        self.chunk_embeddings = None
        self.chunk_metadata = []
        self.is_trained = False
        
        print(f"✅ Initialized RAG system with {embedding_model} on {device}")
    
    def extract_text_from_pdf(self, pdf_path: str, method='pymupdf') -> str:
        """Extract text from PDF file"""
        print(f"📄 Extracting text from: {pdf_path}")
        
        if not os.path.exists(pdf_path):
            raise FileNotFoundError(f"PDF file not found: {pdf_path}")
        
        text = ""
        
        if method == 'pymupdf':
            try:
                # PyMuPDF method - better text extraction
                doc = fitz.open(pdf_path)
                for page_num in range(len(doc)):
                    page = doc.load_page(page_num)
                    text += page.get_text()
                doc.close()
                print(f"✅ Successfully extracted text using PyMuPDF: {len(text)} characters")
            except Exception as e:
                print(f"⚠️ PyMuPDF failed: {e}. Trying PyPDF2...")
                method = 'pypdf2'
        
        if method == 'pypdf2':
            try:
                # PyPDF2 method - fallback
                with open(pdf_path, 'rb') as file:
                    pdf_reader = PyPDF2.PdfReader(file)
                    for page_num in range(len(pdf_reader.pages)):
                        page = pdf_reader.pages[page_num]
                        text += page.extract_text()
                print(f"✅ Successfully extracted text using PyPDF2: {len(text)} characters")
            except Exception as e:
                raise Exception(f"❌ Both PDF extraction methods failed: {e}")
        
        return text
    
    def clean_text(self, text: str) -> str:
        """Clean and preprocess extracted text"""
        # Remove excessive whitespace
        text = re.sub(r'\s+', ' ', text)
        
        # Remove page numbers and headers/footers (basic patterns)
        text = re.sub(r'\n\d+\n', '\n', text)
        text = re.sub(r'\nPage \d+\n', '\n', text)
        
        # Remove special characters but keep medical terminology
        text = re.sub(r'[^\w\s\.\,\;\:\-\(\)]', ' ', text)
        
        # Remove excessive spaces
        text = ' '.join(text.split())
        
        return text.strip()
    
    def create_chunks(self, text: str, custom_chunk_size: Optional[int] = None, 
                     custom_overlap: Optional[int] = None) -> List[Dict]:
        """Split text into chunks with metadata"""
        print("✂️ Creating text chunks...")
        
        # Update text splitter if custom parameters provided
        if custom_chunk_size or custom_overlap:
            self.text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=custom_chunk_size or 1000,
                chunk_overlap=custom_overlap or 200,
                length_function=len,
                separators=["\n\n", "\n", ". ", " ", ""]
            )
        
        # Split text into chunks
        raw_chunks = self.text_splitter.split_text(text)
        
        chunks_with_metadata = []
        for i, chunk in enumerate(raw_chunks):
            if len(chunk.strip()) > 50:  # Filter out very short chunks
                chunk_data = {
                    'text': chunk.strip(),
                    'chunk_id': i,
                    'length': len(chunk),
                    'start_char': text.find(chunk),
                    'type': self._classify_chunk(chunk)
                }
                chunks_with_metadata.append(chunk_data)
        
        print(f"✅ Created {len(chunks_with_metadata)} chunks")
        return chunks_with_metadata
    
    def _classify_chunk(self, chunk: str) -> str:
        """Classify chunk type based on content"""
        chunk_lower = chunk.lower()
        
        if any(word in chunk_lower for word in ['symptom', 'symptoms', 'present', 'criteria']):
            return 'symptoms'
        elif any(word in chunk_lower for word in ['treatment', 'therapy', 'medication', 'intervention']):
            return 'treatment'
        elif any(word in chunk_lower for word in ['diagnosis', 'diagnostic', 'condition', 'disorder']):
            return 'diagnosis'
        else:
            return 'general'
    
    def create_embeddings(self, chunks: List[Dict]) -> np.ndarray:
        """Create embeddings for text chunks"""
        print("🧮 Creating embeddings...")
        
        texts = [chunk['text'] for chunk in chunks]
        
        # Create embeddings in batches to manage memory
        batch_size = 32
        all_embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            batch_embeddings = self.embedder.encode(
                batch_texts,
                batch_size=batch_size,
                show_progress_bar=True if i == 0 else False,
                convert_to_numpy=True
            )
            all_embeddings.append(batch_embeddings)
        
        embeddings = np.vstack(all_embeddings)
        print(f"✅ Created embeddings: {embeddings.shape}")
        
        return embeddings
    
    def build_knowledge_base(self, pdf_path: str, chunk_size: int = 1000, 
                           chunk_overlap: int = 200):
        """Complete pipeline: PDF -> Text -> Chunks -> Embeddings"""
        print("🏗️ Building Knowledge Base...")
        
        # 1. Extract text from PDF
        raw_text = self.extract_text_from_pdf(pdf_path)
        
        # 2. Clean text
        cleaned_text = self.clean_text(raw_text)
        print(f"🧹 Cleaned text length: {len(cleaned_text)} characters")
        
        # 3. Create chunks
        self.chunks = self.create_chunks(cleaned_text, chunk_size, chunk_overlap)
        self.chunk_metadata = [chunk for chunk in self.chunks]
        
        # 4. Create embeddings
        self.chunk_embeddings = self.create_embeddings(self.chunks)
        
        self.is_trained = True
        print("✅ Knowledge base built successfully!")
    
    def save_knowledge_base(self, filepath: str):
        """Save the complete knowledge base"""
        if not self.is_trained:
            raise ValueError("Knowledge base not built. Please build it first.")
        
        kb_data = {
            'chunks': self.chunks,
            'chunk_embeddings': self.chunk_embeddings,
            'chunk_metadata': self.chunk_metadata,
            'embedding_model': self.embedding_model_name,
            'embedder_config': {
                'model_name': self.embedding_model_name,
                'max_seq_length': self.embedder.max_seq_length
            }
        }
        
        with open(filepath, 'wb') as f:
            pickle.dump(kb_data, f)
        print(f"💾 Knowledge base saved to: {filepath}")
    
    def load_knowledge_base(self, filepath: str):
        """Load a pre-built knowledge base"""
        with open(filepath, 'rb') as f:
            kb_data = pickle.load(f)
        
        self.chunks = kb_data['chunks']
        self.chunk_embeddings = kb_data['chunk_embeddings']
        self.chunk_metadata = kb_data['chunk_metadata']
        self.is_trained = True
        
        print(f"📚 Knowledge base loaded from: {filepath}")
        print(f"📊 Loaded {len(self.chunks)} chunks with embeddings")
    
    def search_similar_chunks(self, query: str, top_k: int = 5, 
                            similarity_threshold: float = 0.3) -> List[Dict]:
        """Search for similar chunks using semantic similarity"""
        if not self.is_trained:
            raise ValueError("Knowledge base not built. Please build or load it first.")
        
        # Create query embedding
        query_embedding = self.embedder.encode([query], convert_to_numpy=True)
        
        # Calculate similarities
        similarities = cosine_similarity(query_embedding, self.chunk_embeddings)[0]
        
        # Get top-k results above threshold
        top_indices = np.argsort(similarities)[::-1][:top_k]
        
        results = []
        for idx in top_indices:
            if similarities[idx] >= similarity_threshold:
                result = {
                    'chunk_id': idx,
                    'text': self.chunks[idx]['text'],
                    'metadata': self.chunk_metadata[idx],
                    'similarity_score': float(similarities[idx])
                }
                results.append(result)
        
        return results
    
    def generate_symptom_response(self, query: str, top_k: int = 3) -> Dict:
        """Generate response for symptom query"""
        # Search for relevant chunks
        relevant_chunks = self.search_similar_chunks(query, top_k)
        
        if not relevant_chunks:
            return {
                'response': "I couldn't find relevant information for your symptoms in the knowledge base. Please consult with a healthcare professional for proper diagnosis.",
                'sources': [],
                'confidence': 0.0
            }
        
        # Generate response
        response_parts = []
        
        # Medical disclaimer
        response_parts.append("⚠️ MEDICAL DISCLAIMER: This information is for educational purposes only and is based on clinical literature. Always consult with a qualified healthcare professional for proper diagnosis and treatment.")
        response_parts.append("")
        
        # Main response
        response_parts.append("Based on your symptoms, here's relevant information from the clinical literature:")
        response_parts.append("")
        
        # Add relevant chunks
        for i, chunk in enumerate(relevant_chunks, 1):
            chunk_text = chunk['text']
            # Truncate very long chunks
            if len(chunk_text) > 400:
                chunk_text = chunk_text[:400] + "..."
            
            response_parts.append(f"📄 Source {i} (Similarity: {chunk['similarity_score']:.3f}):")
            response_parts.append(chunk_text)
            response_parts.append("")
        
        # Recommendations
        response_parts.append("🔍 Next Steps:")
        response_parts.append("• Document all symptoms with duration and severity")
        response_parts.append("• Consult with a mental health professional for proper assessment")
        response_parts.append("• Consider keeping a symptom diary")
        response_parts.append("• Seek immediate help if experiencing severe distress")
        
        avg_confidence = np.mean([chunk['similarity_score'] for chunk in relevant_chunks])
        
        return {
            'response': '\n'.join(response_parts),
            'sources': relevant_chunks,
            'confidence': float(avg_confidence),
            'num_sources': len(relevant_chunks)
        }

print("✅ PDFSymptomCheckerRAG class defined successfully!")

# ===== CELL 4: Initialize System =====
# Initialize the RAG system
rag_system = PDFSymptomCheckerRAG(
    embedding_model='all-MiniLM-L6-v2',  # Good balance of speed and quality
    use_gpu=False  # Set to True if you have NVIDIA GPU with CUDA
)

print("🚀 RAG system initialized!")

# ===== CELL 5: Process Your PDF =====
# Update this path to your actual PDF location
pdf_path = r"C:\Users\vreddy_quantum-i\Downloads\DSM-5_filtered.pdf"

try:
    # Build the knowledge base
    rag_system.build_knowledge_base(
        pdf_path=pdf_path,
        chunk_size=1200,    # Larger chunks for clinical content
        chunk_overlap=300   # Good overlap for context preservation
    )
    print("📚 Knowledge base built from PDF!")
    
except FileNotFoundError:
    print(f"❌ PDF file not found at: {pdf_path}")
    print("Please update the path to match your actual file location.")
    print("Example: pdf_path = r'C:\\path\\to\\your\\DSM-5_filtered.pdf'")
except Exception as e:
    print(f"❌ Error processing PDF: {e}")

# ===== CELL 6: Save Knowledge Base =====
if rag_system.is_trained:
    # Save for future use (so you don't have to rebuild)
    kb_file = 'dsm5_symptom_checker.pkl'
    rag_system.save_knowledge_base(kb_file)
    print(f"💾 Knowledge base saved to: {kb_file}")
else:
    print("⚠️ Knowledge base not built. Please fix PDF path and run Cell 5 first.")

# ===== CELL 7: Test Single Query =====
if rag_system.is_trained:
    # Test with a sample symptom query
    test_query = "I feel anxious and have panic attacks"
    response = rag_system.generate_symptom_response(test_query, top_k=3)

    print(f"🔍 Query: {test_query}")
    print(f"📊 Confidence: {response['confidence']:.3f}")
    print(f"📚 Number of sources: {response['num_sources']}")
    print(f"\n📝 Response:\n{response['response']}")
else:
    print("⚠️ Knowledge base not ready. Please build it first.")

# ===== CELL 8: Test Multiple Queries =====
if rag_system.is_trained:
    # Test with multiple queries
    test_queries = [
        "I have trouble concentrating and feel restless",
        "I experience mood swings and feel very sad",
        "I have obsessive thoughts and compulsive behaviors",
        "I hear voices and feel paranoid",
        "I have flashbacks and nightmares after a trauma"
    ]

    print("🧪 Testing multiple queries:\n")
    for i, query in enumerate(test_queries, 1):
        response = rag_system.generate_symptom_response(query, top_k=2)
        print(f"{i}. Query: '{query}'")
        print(f"   📊 Confidence: {response['confidence']:.3f}")
        print(f"   📚 Sources: {response['num_sources']}")
        print("-" * 60)
else:
    print("⚠️ Knowledge base not ready. Please build it first.")

# ===== CELL 9: Custom Query Function =====
def ask_symptom_checker(symptoms, top_k=3):
    """
    Convenience function for quick queries
    
    Args:
        symptoms (str): Description of symptoms
        top_k (int): Number of sources to retrieve
    
    Returns:
        Formatted response
    """
    if not rag_system.is_trained:
        return "❌ Knowledge base not loaded. Please run the setup cells first."
    
    response = rag_system.generate_symptom_response(symptoms, top_k)
    
    print(f"🔍 Query: {symptoms}")
    print(f"📊 Confidence: {response['confidence']:.3f}")
    print(f"📚 Sources: {response['num_sources']}")
    print(f"\n{response['response']}")
    
    return response

print("✅ Helper functions defined!")

# Example usage (uncomment to test):
# ask_symptom_checker("I feel depressed and have no energy")

# ===== CELL 10: System Statistics =====
if rag_system.is_trained:
    print("📊 Knowledge Base Statistics:")
    print(f"• Total chunks: {len(rag_system.chunks)}")
    print(f"• Embedding dimensions: {rag_system.chunk_embeddings.shape[1]}")
    print(f"• Model: {rag_system.embedding_model_name}")
    print(f"• Device: {'GPU' if rag_system.use_gpu else 'CPU'}")
    
    # Show chunk types
    chunk_types = {}
    for chunk in rag_system.chunks:
        chunk_type = chunk.get('type', 'general')
        chunk_types[chunk_type] = chunk_types.get(chunk_type, 0) + 1
    
    print(f"• Chunk distribution: {chunk_types}")
    
    # Show some sample chunks
    print(f"\n📋 Sample chunks:")
    for i, chunk in enumerate(rag_system.chunks[:3], 1):
        preview = chunk['text'][:100] + "..." if len(chunk['text']) > 100 else chunk['text']
        print(f"{i}. [{chunk['type']}] {preview}")
else:
    print("⚠️ Knowledge base not built yet. Please run the setup cells first.")

# ===== CELL 11: Load Existing Knowledge Base (For Future Sessions) =====
"""
# For future sessions, you can load pre-built knowledge base:

# Initialize new system
rag_system_loaded = PDFSymptomCheckerRAG()

# Load existing knowledge base
rag_system_loaded.load_knowledge_base('dsm5_symptom_checker.pkl')

# Now you can immediately start querying
response = rag_system_loaded.generate_symptom_response("your symptoms here")
print(response['response'])
"""

print("✅ All cells ready!")
print("💡 Instructions:")
print("1. Update the PDF path in Cell 5 to match your file location")
print("2. Run cells sequentially from Cell 1 to Cell 10")
print("3. Use ask_symptom_checker('your symptoms') for quick queries")
print("4. For future sessions, use the code in Cell 11 to load existing knowledge base")

✅ All imports successful!
✅ PDFSymptomCheckerRAG class defined successfully!
Loading embedding model: all-MiniLM-L6-v2 on cpu
✅ Initialized RAG system with all-MiniLM-L6-v2 on cpu
🚀 RAG system initialized!
🏗️ Building Knowledge Base...
📄 Extracting text from: C:\Users\vreddy_quantum-i\Downloads\DSM-5_filtered.pdf
✅ Successfully extracted text using PyMuPDF: 3097485 characters
🧹 Cleaned text length: 3092736 characters
✂️ Creating text chunks...
✅ Created 3451 chunks
🧮 Creating embeddings...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Created embeddings: (3451, 384)
✅ Knowledge base built successfully!
📚 Knowledge base built from PDF!
💾 Knowledge base saved to: dsm5_symptom_checker.pkl
💾 Knowledge base saved to: dsm5_symptom_checker.pkl
🔍 Query: I feel anxious and have panic attacks
📊 Confidence: 0.579
📚 Number of sources: 3

📝 Response:
⚠️ MEDICAL DISCLAIMER: This information is for educational purposes only and is based on clinical literature. Always consult with a qualified healthcare professional for proper diagnosis and treatment.

Based on your symptoms, here's relevant information from the clinical literature:

📄 Source 1 (Similarity: 0.588):
. The course of panic disorder typically is complicated by a range of other disorders, in Temperamental. Environmental. particular other anxiety disorders, depressive disorders, and substance use disorders (see section Comorbidity for this disorder). African American adults have been reported to have a more chronic course of panic disorder compared with non-Latinx Whit